<a href="https://colab.research.google.com/github/ahmednbail/ahmednbail/blob/main/parse_model__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 899.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 32.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [2]:

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json



In [3]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 12.58 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [4]:
cv_data=json.load(open("/content/CV-Parsing-using-Spacy-3/data/training/train_data.json",'r'))

In [5]:
len(cv_data)

200

In [6]:
!python -m spacy init fill-config  /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
def get_spacy_doc(file,data):
  nlp=spacy.blank('en')
  db=DocBin()


  for text , annot in tqdm(data):

    doc=nlp.make_doc(text)
    annot = annot['entities']


    ents = []
    entity_indices = []

    for start  , end , label in annot:
      skip_entity=False
      for idx in range(start , end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices  + list(range(start,end))

      try:
        span = doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue
      if span is None :
        err_data=str([start,end]) + "     " + str(text)  + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db













In [8]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(cv_data,test_size=0.3)

In [9]:

file=open('error.txt','w')
db =get_spacy_doc(file,train)
db.to_disk("train_data.spacy")

db =get_spacy_doc(file,test)
db.to_disk("test_data.spacy")

file.close()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 60/60 [00:00<00:00, 85.48it/s]


In [10]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 163kB/s]
config.json: 100% 481/481 [00:00<00:00, 3.43MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 12.4MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 5.06MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 27.2MB/s]
model.safetensors: 100% 499M/499M [00:02<00:00, 197MB/s]
Some weights of R

In [11]:
nlp=spacy.load('/content/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [12]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 35.5 MB/s eta 0:00:00


In [13]:
import sys ,fitz

In [14]:
fname='/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'
doc=fitz.open(fname)

In [15]:
text= ''
for page in doc:
   text=text +str(page.get_text())

In [16]:
text =text.strip()

In [17]:

text="  ".join(text.split())

In [18]:
text

'Alice  Clark  AI  /  Machine  Learning  Delhi,  India  Email  me  on  Indeed  •  20+  years  of  experience  in  data  handling,  design,  and  development  •  Data  Warehouse:  Data  analysis,  star/snow  flake  scema  data  modelling  and  design  specific  to  data  warehousing  and  business  intelligence  •  Database:  Experience  in  database  designing,  scalability,  back-up  and  recovery,  writing  and  optimizing  SQL  code  and  Stored  Procedures,  creating  functions,  views,  triggers  and  indexes.  Cloud  platform:  Worked  on  Microsoft  Azure  cloud  services  like  Document  DB,  SQL  Azure,  Stream  Analytics,  Event  hub,  Power  BI,  Web  Job,  Web  App,  Power  BI,  Azure  data  lake  analytics(U-SQL)  Willing  to  relocate  anywhere  WORK  EXPERIENCE  Software  Engineer  Microsoft  –  Bangalore,  Karnataka  January  2000  to  Present  1.  Microsoft  Rewards  Live  dashboards:  Description:  -  Microsoft  rewards  is  loyalty  program  that  rewards  Users  for

In [19]:
doc=nlp(text)
for ent in doc.ents:
  print(ent.label_, "    ->>>>>>",ent.text)

Location     ->>>>>> Delhi
Designation     ->>>>>> Software  Engineer
Companies worked at     ->>>>>> Microsoft
College Name     ->>>>>> Indian  Institute  of  Technology
Skills     ->>>>>> Machine  Learning,  Natural  Language  Processing,  and  Big  Data  Handling  ADDITIONAL  INFORMATION  Professional  Skills  •  Excellent  analytical,  problem  solving,  communication,  knowledge  transfer  and  interpersonal  skills  with  ability  to  interact  with  individuals  at  all  the  levels  


In [20]:
def Parse_cv(Cv):
  import sys ,fitz
  doc=fitz.open(Cv)
  text= ''
  for page in doc:
     text=text +str(page.get_text())

  text =text.strip()
  text="  ".join(text.split())

  doc=nlp(text)
  for ent in doc.ents:
    Parse= print(ent.label_, "    ->>>>>>",ent.text)

  return Parse





In [21]:
Parse_cv('/content/CV-Parsing-using-Spacy-3/data/test/Smith Resume.pdf')

Name     ->>>>>> Michael  
Email Address     ->>>>>> indeed.com/r/falicent/140749dace5dc26f
Designation     ->>>>>> Engineer
College Name     ->>>>>> University  of  Manchester
Graduation Year     ->>>>>> 2007
Skills     ->>>>>> problem  solving  (Less  than  1  year),  project  lifecycle  (Less  than  1  year),  project  manager  (Less  than  1  year),  technical  assistance.  (Less  than  1  year)
